In [1]:
import os
import numpy as np
import pandas as pd
import glob
import cv2
import shutil

In [2]:
## to check if ffmpeg is installed
import subprocess

def is_ffmpeg_installed():
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

# Usage
if is_ffmpeg_installed():
    print("FFmpeg is installed.")
else:
    print("FFmpeg is not installed.")

FFmpeg is installed.


In [3]:
def createVideo():
    import subprocess
    import os

    # FFmpeg command
    ffmpeg_cmd = [
        "ffmpeg",
        "-framerate", str(frame_rate),
        "-start_number", str(start_frame),
        "-i", image_pattern,
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        output_video
    ]
    
    
    # Run the command  
    try:
        result = subprocess.run(ffmpeg_cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print("Error during FFmpeg execution:")
        print(e.stderr)  # This shows the real error from FFmpeg

In [4]:
# information about the videos that are to be masked - use datasheet on Drive - csv file
seq_info = r"G:\.shortcut-targets-by-id\11AE4tEztgQ4SLqv4eoIUjjTJQvPkhRNy\MuscaChase\Musca-Musca_Chase_AnimalChart_experimentNotes.csv"
temp = pd.read_csv(seq_info)
videos = temp[(temp.BodyMarked.notnull())]
videos

,Filename,Date of experiment,Framerate,No. of intact males in the box,No. of ablated males in the box,No. of females in the box,Marking on the ablated males,Marking on the females,Date of collection of flies,Cameras used,...,Digitizer,Digitization status,BodyMarked,"start, stop on DLTdv","start frame number, cine video","end frame number, cine video",difference,DURATION OF CHASE,Caught?,Unnamed: 24
81,2023-04-19_Trial1_Set1_5k_16024.cine,2023-04-19,5000,10,10,5,Horizontal Line,Dot,2023-04-19,16024_20654_20655,...,NaN,NaN,"Yes, Tanvi","70, 786",already chasing when filming began,-10362.0,#VALUE!,#VALUE!,yes (II),NaN


In [5]:
cam1_path = r"D:\Musca Chase\2023-04-19_Musca chase_Highspeed\\2023-04-19_Trial1_Set1_5k_16024.mp4"
cam2_path = r"D:\Musca Chase\2023-04-19_Musca chase_Highspeed\2023-04-19_Trial1_Set1_5k_20654.mp4"
cam3_path = r"D:\Musca Chase\2023-04-19_Musca chase_Highspeed\2023-04-19_Trial1_Set1_5k_20655.mp4"
camlist = [cam1_path, cam2_path, cam3_path]

DLTxypoints_path = r"G:\.shortcut-targets-by-id\11AE4tEztgQ4SLqv4eoIUjjTJQvPkhRNy\MuscaChase\MS_Analysis\MuscaChasingMusca\preprocessing_image\\2023_04_19-Trial1_Set1_5kxypts.csv"

In [6]:
# create temp_dir
temp_dir = r"../../../MuscaChase/dataFolders/MuscaChasingMusca/Preprocessing_Images/temp/"

In [ ]:
for ii, cam in enumerate(camlist):
    # read the video
    cap = cv2.VideoCapture(cam)

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
  
    # get the frame start and stop in the video
    Frames = videos[videos.Filename == [name for name in videos.Filename if cam.split("\\")[-1][:-9] in name][0]]
    start = int(Frames['start on DLTdv'].values[0])
    stop = int(Frames['stop on DLTdv'].values[0])
    # Specify range
    start_frame = start
    end_frame = stop
    
    # get the coordinates for transparent rectangle
    coords = pd.read_csv(DLTxypoints_path)
    subset = coords.loc[start_frame:end_frame, :]
    
    cam_subset = subset.loc[:,[columns for columns in subset.columns if 'cam' + str(ii+1) in columns]]  
    chaser =  cam_subset.iloc[:, 0:2]
    chasee = cam_subset.iloc[:, 2:4]
    
    # create temp_dir
    os.mkdir(temp_dir)
    print(f'working with {cam}')
    while cap.isOpened():
              
        for current_frame in np.arange(start_frame, end_frame):       
            # Set the frame to be read
            cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
            ret, frame = cap.read()
            if not ret:
                print(f'did not read frame {current_frame}')
                break

            #frame number
#             print(current_frame)

            ######### Work with chaser images ############################################
            # Create black background
            black_frame = np.zeros_like(frame)
            # Parameters for transparent rectangle
            position = chaser.loc[current_frame,:]
            
            # if fly is present (not a NaN)
            if not position.isnull().values.any():
#                 if ii == 0: # ii == 16024 camera, use these lines to flip the x axis
#                     xpos = frame_width - int(position[0])
#                 else:
#                     xpos = int(position[0])
                xpos = int(position[0]) # comment this line if using the if loop above
                
                ypos = int(position[1])
                x, y = xpos - 25, ypos - 25    # Top-left corner of transparent area
                dt = 50 # Width and height of the transparent rectangle

                # Copy only the transparent rectangle from original frame
                black_frame[y:y+dt, x:x+dt] = frame[y:y+dt, x:x+dt]

            # Write the masked frame as image
            frame_filename = os.path.join(temp_dir, f"chaser_frame_{current_frame:05d}.png")
            cv2.imwrite(frame_filename, black_frame)
            
            
            # Work with chasee images
            # Create black background
            black_frame = np.zeros_like(frame)
            # Parameters for transparent rectangle
            position = chasee.loc[current_frame,:]
            
            # if fly is present (not a NaN)
            if not position.isnull().values.any():
#                 if ii == 0: # ii == 16024 camera, use these lines to flip the x axis
#                     xpos = frame_width - int(position[0])
#                 else:
#                     xpos = int(position[0])
                
                xpos = int(position[0]) # comment this line if using the if loop above
                ypos = int(position[1])
                x, y = xpos - 25, ypos - 25    # Top-left corner of transparent area
                dt = 50 # Width and height of the transparent rectangle

                # Copy only the transparent rectangle from original frame
                black_frame[y:y+dt, x:x+dt] = frame[y:y+dt, x:x+dt]

            # Write the masked frame as image
            frame_filename = os.path.join(temp_dir, f"chasee_frame_{current_frame:05d}.png")
            cv2.imwrite(frame_filename, black_frame)
            
            
        cap.release()
        
    # get the output video name
    outdir = r"E:\Tanvi Deora\Documents\GitHub\MuscaChase\dataFolders\MuscaChasingMusca\Preprocessing_Images"
    frame_rate = fps
    image_folder = temp_dir

    # chaser Parameters
    image_pattern = os.path.join(image_folder, "chaser_frame_%05d.png")
    outfilename = cam.split("\\")[-1][:-4] + '_chaser.mp4'
    output_video = os.path.join(outdir, outfilename)
    createVideo()

    # chasee Parameters
    image_pattern = os.path.join(image_folder, "chasee_frame_%05d.png")
    outfilename = cam.split("\\")[-1][:-4] + '_chasee.mp4'
    output_video = os.path.join(outdir, outfilename)
    createVideo()
    
    shutil.rmtree(temp_dir)

working with D:\Musca Chase\2023-04-19_Musca chase_Highspeed\\2023-04-19_Trial1_Set1_5k_16024.mp4


working with D:\Musca Chase\2023-04-19_Musca chase_Highspeed\2023-04-19_Trial1_Set1_5k_20654.mp4


working with D:\Musca Chase\2023-04-19_Musca chase_Highspeed\2023-04-19_Trial1_Set1_5k_20655.mp4


